In [ ]:
import torch

In [13]:
# utils is at ../utils
import sys
import os

# Add the parent directory (src) to the path so we can import from utils
sys.path.append(os.path.join(os.getcwd(), ".."))
from utils.general_utils import hello_world

In [14]:
hello_world()

Hello World


In [2]:
available_devices = ["cuda" if torch.cuda.is_available() else "cpu"]
if torch.backends.mps.is_available():
    available_devices.append("mps")
print("Available devices:", available_devices)


Available devices: ['cpu', 'mps']
